# Gapfillling

GrowMatch and SMILEY are gap-filling algorithms, which try to to make the minimal number of changes to a model and allow it to simulate growth. For more information, see [Kumar et al.](http://dx.doi.org/10.1371/journal.pcbi.1000308). Please note that these algorithms are Mixed-Integer Linear Programs, which need solvers such as gurobi or cplex to function correctly.

In [1]:
import cobra.test

model = cobra.test.create_test_model("salmonella")

In this model D-Fructose-6-phosphate is an essential metabolite. We will remove all the reactions using it, and at them to a separate model.

In [2]:
# remove some reactions and add them to the universal reactions
Universal = cobra.Model("Universal_Reactions")
for i in [i.id for i in model.metabolites.f6p_c.reactions]:
    reaction = model.reactions.get_by_id(i)
    Universal.add_reaction(reaction.copy())
    reaction.remove_from_model()

Now, because of these gaps, the model won't grow.

In [3]:
model.optimize().f

2.821531499799383e-12

## GrowMatch

We will use GrowMatch to add back the minimal number of reactions from this set of "universal" reactions (in this case just the ones we removed) to allow it to grow.

In [4]:
r = cobra.flux_analysis.growMatch(model, Universal)
for e in r[0]:
    print(e.id)

GF6PTA
FBP
MAN6PI_reverse
TKT2_reverse
PGI_reverse


We can obtain multiple possible reaction sets by having the algorithm go through multiple iterations.

In [5]:
result = cobra.flux_analysis.growMatch(model, Universal,
                                       iterations=4)
for i, entries in enumerate(result):
    print("---- Run %d ----" % (i + 1))
    for e in entries:
        print(e.id)

---- Run 1 ----
GF6PTA
FBP
MAN6PI_reverse
TKT2_reverse
PGI_reverse
---- Run 2 ----
F6PP
GF6PTA
TALA
MAN6PI_reverse
F6PA_reverse
---- Run 3 ----
GF6PTA
MAN6PI_reverse
TKT2_reverse
F6PA_reverse
PGI_reverse
---- Run 4 ----
F6PP
GF6PTA
FBP
TALA
MAN6PI_reverse


## SMILEY

SMILEY is very similar to growMatch, only instead of setting growth as the objective, it sets production of a specific metabolite

In [6]:
r = cobra.flux_analysis.gapfilling.SMILEY(model, "ac_e",
                                          Universal)
for e in r[0]:
    print(e.id)

GF6PTA
MAN6PI_reverse
TKT2_reverse
F6PA_reverse
PGI_reverse
